# 👨‍💼👨‍🚀👨‍💻 Agents with Personas

### Initialize the kernel

In [ ]:
#r "nuget: Microsoft.SemanticKernel, 1.18.0-rc"
#r "nuget: Microsoft.SemanticKernel.Plugins.Core, 1.18.0-alpha"
#r "nuget: Microsoft.SemanticKernel.Plugins.Web, 1.18.0-alpha"
#r "nuget: Microsoft.SemanticKernel.Agents.Core, 1.18.0-alpha"

#!import ../../Secrets.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.SemanticKernel.Agents;
using Microsoft.SemanticKernel.Agents.Chat;
using Microsoft.SemanticKernel.ChatCompletion;

var kernel = Kernel.CreateBuilder()
    .AddAzureOpenAIChatCompletion(
        deploymentName: Secrets.DeploymentName,
        endpoint: Secrets.Endpoint,
        apiKey: Secrets.ApyKey)
    //.AddOpenAIChatCompletion(
    //    modelId: Secrets.OpenAIModel,
    //    apiKey: Secrets.OpenAIApiKey)    
    .Build();

### 🧱 A persona is the behaviour we give to an agent. Our newly created agent gets the behavior of Jack Sparrow pirate.

In [ ]:
#pragma warning disable SKEXP0110

ChatCompletionAgent jackSparrowAgent = new() 
{ 
    Kernel = kernel,
    Instructions = """
        You are Jack Sparrow talking in Jack Sparrow style.
        Evaluate the context and reply providing exactly one meaningful dialog line.
        The dialog line must be only one sentence of maximum 10 words.
        """,
    Description = "A chat bot that replies to the message in the voice of Jack Sparrow talking style.",
    Name = "JackSparrow",
    Id = "JackSparrow_01",
};

### Let's create a chat group for agents and add a goal to the chat

In [ ]:
#pragma warning disable SKEXP0110

AgentGroupChat chat = new();

//ChatHistory x;

var goal = """
    Jack Sparrow makes a bad joke about Don Quijote's taste in drinks.
    Jack Sparrow gets a threat from Don Quijote and Don Quijote is launching a fake attack.
    """;

// actions: Jack makes a bad joke, Don Quijote threats Jack, Don Quijote is launching a fake attach

chat.AddChatMessage(new ChatMessageContent(AuthorRole.User, goal));

Console.WriteLine($">>>>>>> {AuthorRole.User}: {goal}");


In [ ]:
//x.AddUserMessage(goal);
//x.InvokeAsync(jackSparrowAgent);

### ⚡ The chat is invoking Jack Sparrow agent on its history

In [ ]:
#pragma warning disable SKEXP0001
#pragma warning disable SKEXP0110

// jackSparrow agent is reacting to the chat history
await foreach (var content in chat.InvokeAsync(jackSparrowAgent))
{
    Console.WriteLine($"{content.Role} [{content.AuthorName}] > {content.Content}");
}

### 🧱 Let's make another agent. This one impersonates Don Quijote.

In [ ]:
#pragma warning disable SKEXP0110

ChatCompletionAgent donQuijoteAgent = new() 
{ 
    Arguments = new KernelArguments (new OpenAIPromptExecutionSettings { ToolCallBehavior = ToolCallBehavior.AutoInvokeKernelFunctions }),
    Kernel = kernel,
    Instructions = """
        You are Don Quijote talking in Don Quijote style.
        Evaluate the context and reply to the last message by providing exactly one meaningful dialog line.
        The dialog line must be only one sentence of maximum 10 words.
        """,
    Description = "A chat bot that replies to the message in the voice of Don Quijote talking style.",
    Name = "DonQuijote",
    Id = "DonQuijote_01",
};

### ⚡ The chat is invoking Don Quijote agent on its history

In [ ]:
#pragma warning disable SKEXP0001
#pragma warning disable SKEXP0110

// jackSparrow agent is reacting to the chat history
await foreach (var content in chat.InvokeAsync(donQuijoteAgent))
{
    Console.WriteLine($"{content.Role} [{content.AuthorName}] > {content.Content}");
}


In [ ]:
#pragma warning disable SKEXP0110

Console.WriteLine("Chat history: ");
await foreach (var m in chat.GetChatMessagesAsync())
{
    Console.WriteLine($"[{m.Role}] {m.Content}");
}

### Notice that the chat history is having the responses from both agents.